# The code transforms an excel survey into a word survey.

In [3]:
#! pip install python-docx
import pandas as pd
import numpy as np
from docx import Document
#!pip install docxtpl
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.shared import Pt, RGBColor, Inches
#from docxtpl import DocxTemplate

In [4]:
df = pd.ExcelFile('/Users/sedastepanyan/Downloads/household_questionnaire_geopoll_EN_template_20230720_ISO3 (2).xlsx')

In [5]:
df.sheet_names

['survey', 'New Durations', 'Crop list', 'Additional information']

In [6]:
#This code converts scientific notation to numeric notation throughout the whole code
#pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [7]:
#Parse the sheet to generate a dataframe
df=df.parse('survey')
list(df.columns)

['Q#',
 'Q Name',
 'Suggested Qname',
 'English',
 'Length',
 'Q Type',
 'Randomize',
 'Conditional',
 'Programming Instructions',
 'Skip Pattern',
 'Codes',
 'Default skip patterns & conditional ',
 'Specify skip pattern variable (from blue text)',
 'Unnamed: 13',
 'Estimated percentage of repondents',
 'Unnamed: 15',
 'Unnamed: 16',
 'Mandatory',
 'Unnamed: 18',
 'Unnamed: 19',
 'Unnamed: 20',
 'Duration (sec)',
 'Weighted duration',
 'Core questions only']

In [8]:
#doc = Document()
#doc.styles['Normal'].font.name = 'Arial'

In [9]:
#df.drop(df.iloc[[2, 19, 40, 57, 99, 128, 136, 150, 158, 169, 177, 228, 237]].index)


In [10]:
#df = df[df.iloc[:, 0] != 'NA']
#df

In [11]:
df.dropna(subset = ['Q Name'], inplace = True)
df

,Q#,Q Name,Suggested Qname,English,Length,Q Type,Randomize,Conditional,Programming Instructions,Skip Pattern,...,Estimated percentage of repondents,Unnamed: 15,Unnamed: 16,Mandatory,Unnamed: 18,Unnamed: 19,Unnamed: 20,Duration (sec),Weighted duration,Core questions only
1,NaN,Optin,NaN,Optin,5.0,Open Ended,NaN,NaN,Form Type - TextBox,Any Response = phone_number,...,NaN,NaN,NaN,no,NaN,NaN,NaN,0.000000,0.000000,NaN
2,NaN,phone_number,phone_number,Input the phone number that you are contacting:,48.0,Open Ended,NaN,NaN,Form Type - TextBox,Any Response = calldispo,...,NaN,NaN,NaN,yes,NaN,NaN,NaN,0.000000,0.000000,0.000000
3,NaN,calldispo,calldispo,Phone number: #CATI_MOBILENUMBER#\n1)Someone a...,146.0,Single Choice,NaN,NaN,Form Type - Buttons,1 = resp_language\n2 = AnsweringMachine\n3 = N...,...,NaN,NaN,NaN,no,NaN,NaN,NaN,14.695308,0.000000,NaN
4,NaN,AnsweringMachine,calldispo_answeringmachine,Reached an answering machine.,29.0,NaN,NaN,NaN,NaN,End Poll AnsweringMachine,...,NaN,NaN,NaN,no,NaN,NaN,NaN,0.000000,0.000000,NaN
5,NaN,NoAnswer,calldispo_noanswer,Phone rang and no one answered.,31.0,NaN,NaN,NaN,NaN,End Poll NoAnswer,...,NaN,NaN,NaN,no,NaN,NaN,NaN,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,NaN,callback,callback,Your answers will help us to understand and re...,162.0,Single Choice,NaN,NaN,Form Type - Buttons,1 = leader_contact\n2 = language2,...,1.0,NaN,NaN,yes,NaN,NaN,NaN,14.946465,14.946465,14.946465
237,NaN,future_int_day,future_int_day_\n\n1) future_int_day_mon\n2) f...,"For future interviews, in which day[s] can I c...",166.0,Select All That Apply,NaN,NaN,Form Type - Checklist,1-7 = future_int_time,...,1.0,NaN,NaN,yes - panel,NaN,NaN,NaN,19.354914,19.354914,NaN
238,NaN,future_int_time,future_int_time_ \n\n1) future_int_time_morn\n...,"For future interviews, at what time[s] can I c...",130.0,Select All That Apply,NaN,NaN,Form Type - Checklist,1-3 = language2,...,1.0,NaN,NaN,yes - panel,NaN,NaN,NaN,15.889400,15.889400,NaN
240,NaN,language2,language2,Select the language that was mostly used to co...,158.0,Single Choice,NaN,NaN,Form Type - Buttons,1-2 = closeout,...,1.0,NaN,NaN,yes,NaN,NaN,NaN,17.309154,17.309154,17.309154


In [12]:
# Add sequential numbers to the 'Q#' column where it is NA
df['Q#'] = range(1, len(df) + 1)
df['Q#'] = 'Q' + df['Q#'].astype(str)


In [16]:

# Create a Word document
doc = Document()

# Loop through each row in the Excel data
for index, row in df.iterrows():
    question_number = row['Q#']
    question_name = row['Q Name']
    programming_instructions = row['Programming Instructions']
    conditional = row['Conditional']
    question_title = row['English']
    skip_pattern = row['Skip Pattern']

    # Add Question Number and Question Name
    doc.add_heading(f'{question_number} - {question_name}', level=1)

    # Add Conditional
    if not pd.isnull(conditional):
        doc.add_paragraph(f'Conditional: {conditional}', style='Normal')

    # Add Programming Instructions
    if not pd.isnull(programming_instructions):
        doc.add_paragraph('[' + programming_instructions + ']', style='Normal')

    # Add Question Title (Indented)
    question_title_paragraph = doc.add_paragraph('', style='Heading2')
    question_title_run = question_title_paragraph.add_run(question_title)
    question_title_run.bold = True
    question_title_run.font.size = Pt(12)  # Adjust the font size as needed
    question_title_run.font.color.rgb = RGBColor(0, 0, 0)  # Set color to black
    question_title_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
    question_title_paragraph.style.paragraph_format.left_indent = Inches(0.5)  # Adjust the indent as needed

    # Add Options (if available)
    options_added = False
    for column in df.columns:
        if column.startswith('Option'):
            option = row[column]
            skip_to = row[column.replace('Option', 'Skip')]
            if option:
                option_text = f'{option}\n[Skip Pattern]\n{skip_to}' if skip_to else option
                doc.add_paragraph(option_text, style='ListBullet')
                options_added = True

    # Add Skip Pattern
    if not pd.isnull(skip_pattern):
        doc.add_heading('Skip Pattern', level=3)
        doc.add_paragraph(str(skip_pattern), style='Normal')

    # Add spacing between sections
    if options_added or not pd.isnull(skip_pattern):
        doc.add_paragraph('', style='Normal')

# Save the Word document
output_file = 'output_word_document.docx'
doc.save(output_file)

print(f'Word document "{output_file}" has been created.')


Word document "output_word_document.docx" has been created.
